# Ubicaciones.xlsx -> (ubi_cliente)

In [12]:
import re
import numpy as np
import logging
import pandas as pd
import openpyxl

import warnings
warnings.filterwarnings('ignore')
import unicodedata

In [13]:
%cd "C:\Users\Administrador.PUESTO09\Desktop\Mapas_original\data\Sucia"

C:\Users\Administrador.PUESTO09\Desktop\Mapas_original\data\Sucia


In [15]:
# 1. Leer el archivo Excel

df_ubicaciones = pd.read_excel("Ubicaciones.xlsx", sheet_name='Ubicacion', engine='openpyxl')
print("Datos leídos con éxito. Muestra de datos antes de la limpieza:")
df_ubicaciones.info()
df_ubicaciones.head(2)

Datos leídos con éxito. Muestra de datos antes de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3127 entries, 0 to 3126
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     746 non-null    float64       
 1   CODIGO2    745 non-null    object        
 2   UBICACIÓN  745 non-null    object        
 3   FECHA      746 non-null    datetime64[ns]
 4   ORIGEN     746 non-null    object        
 5   CODCLI2    3127 non-null   int64         
 6   TAREAS     2878 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 171.1+ KB


,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS
0,11941.0,"11941, LO DE RAMON",https://maps.app.goo.gl/VjiuoJ242o6dvV1t9,2025-03-03,agregados_adrian,11941,11941.0
1,80318.0,"80318,MINIMARKET LA 10",https://maps.app.goo.gl/Sy2wPvASNY61HVzv5,2025-03-03,agregados_adrian,80318,80318.0


In [16]:
# Dejo solo los que tienen ubicaciones. 

df_ubicaciones = df_ubicaciones.dropna(subset=['UBICACIÓN'])

In [17]:
df_ubicaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 745 entries, 0 to 745
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     745 non-null    float64       
 1   CODIGO2    744 non-null    object        
 2   UBICACIÓN  745 non-null    object        
 3   FECHA      745 non-null    datetime64[ns]
 4   ORIGEN     745 non-null    object        
 5   CODCLI2    745 non-null    int64         
 6   TAREAS     497 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 46.6+ KB


## BLOQUE 2:  PROCESAMIENTO DE NOMBRES Y CÓDIGOS 

**Observaciones df_ubicaciones** 
- La columna CODIGO2, incluye nombres, vamos a dejarlo con el nombre NOMCLI.
- La columna UBICACION contiene los enlaces con coordenadas a Google Map podriamos proceder creando una columna Latitud y otra Longitud.
  
**Procedemos a ejecutar lo descrito:**

In [18]:
df_ubicaciones.rename(columns={'CODIGO2': 'NOMCLI'}, inplace=True)

## BLOQUE 3: PROCESAMIENTO DE UBICACIONES

In [19]:
def extract_coordinates(ubicacion):
    if pd.isna(ubicacion):
        return pd.Series({
            'LATITUD': None, 
            'LONGITUD': None, 
            'RAZON_COORDENADAS': 'Ubicación nula'
        })

    ubicacion = str(ubicacion)
    patterns = [
        r'lat/lng:\s*\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)',
        r'lat/lng:\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'http[s]?://maps\.google\.com/\?q\s*=\s*(-?\d+\.\d+)\s*,\s*(-?\d+\.\d+)',
        r'@(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'q=(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'!3d\s*(-?\d+\.\d+)!4d\s*(-?\d+\.\d+)',
        r'https?://maps\.app\.goo\.gl/[^?]*\?q=(-?\d+\.\d+),\s*(-?\d+\.\d+)',
    ]

    for pattern in patterns:
        match = re.search(pattern, ubicacion)
        if match:
            try:
                lat = float(match.group(1))
                lon = float(match.group(2))
                return pd.Series({
                    'LATITUD': lat, 
                    'LONGITUD': lon, 
                    'RAZON_COORDENADAS': 'Coordenadas extraídas'
                })
            except ValueError:
                return pd.Series({
                    'LATITUD': None, 
                    'LONGITUD': None, 
                    'RAZON_COORDENADAS': 'Error en conversión de coordenadas'
                })

    # Búsqueda de coordenadas adicional
    all_coords = re.findall(r'(-?\d+\.\d+)', ubicacion)
    if len(all_coords) >= 2:
        try:
            lat = float(all_coords[0])
            lon = float(all_coords[1])
            return pd.Series({
                'LATITUD': lat, 
                'LONGITUD': lon, 
                'RAZON_COORDENADAS': 'Coordenadas alternativas extraídas'
            })
        except ValueError:
            pass

    return pd.Series({
        'LATITUD': None, 
        'LONGITUD': None, 
        'RAZON_COORDENADAS': 'No se encontraron coordenadas'
    })

# Proceso de extracción
df_ubicaciones[['LATITUD', 'LONGITUD', 'RAZON_COORDENADAS']] = df_ubicaciones['UBICACIÓN'].apply(extract_coordinates)

# Análisis de resultados
print("Resumen de extracción de coordenadas:")
print(df_ubicaciones['RAZON_COORDENADAS'].value_counts())

# Exportar filas problemáticas
filas_problematicas = df_ubicaciones[df_ubicaciones['LATITUD'].isna()]
filas_problematicas.to_excel('ubicaciones_sin_coordenadas.xlsx', index=False)

Resumen de extracción de coordenadas:
RAZON_COORDENADAS
Coordenadas extraídas            732
No se encontraron coordenadas     13
Name: count, dtype: int64


In [21]:
filas_problematicas.head(15)

,CODIGO,NOMCLI,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS,LATITUD,LONGITUD,RAZON_COORDENADAS
0,11941.0,"11941, LO DE RAMON",https://maps.app.goo.gl/VjiuoJ242o6dvV1t9,2025-03-03,agregados_adrian,11941,11941.0,NaN,NaN,No se encontraron coordenadas
1,80318.0,"80318,MINIMARKET LA 10",https://maps.app.goo.gl/Sy2wPvASNY61HVzv5,2025-03-03,agregados_adrian,80318,80318.0,NaN,NaN,No se encontraron coordenadas
2,80324.0,"80324,KIOSCO FANTASIA",https://maps.app.goo.gl/fcohXCZ8BQbjL6J96,2025-03-03,agregados_adrian,80324,80324.0,NaN,NaN,No se encontraron coordenadas
3,80317.0,"80317,LA MONTAÑESA",https://maps.app.goo.gl/fHwvyXkPr931h7qv5,2025-03-03,agregados_adrian,80317,80317.0,NaN,NaN,No se encontraron coordenadas
4,13350.0,"13350,ERIKSU",https://maps.app.goo.gl/fVrSz7KgzgQ3jsGTA,2025-03-03,agregados_adrian,13350,13350.0,NaN,NaN,No se encontraron coordenadas
5,80319.0,"80319,ALMACEN QUIBOW",https://maps.app.goo.gl/NQHa6DqB9v9YgpdV6,2025-03-03,agregados_adrian,80319,80319.0,NaN,NaN,No se encontraron coordenadas
6,62619.0,"62619, LETICIA COSTA",https://maps.app.goo.gl/JhiEkkF7XM52CKQy8,2025-02-04,agregados_adrian,62619,NaN,NaN,NaN,No se encontraron coordenadas
161,100006.0,CASA LAUTARO,https://maps.app.goo.gl/RdKDZfi5WDGPKMkt9,2024-09-05,agregados_adrian,100006,NaN,NaN,NaN,No se encontraron coordenadas
194,100000.0,EMPRESA,https://maps.app.goo.gl/TJdD7rTMyTRzzCbY7,2024-08-07,agregados_adrian,100000,NaN,NaN,NaN,No se encontraron coordenadas
195,100001.0,CASA DARIO,https://maps.app.goo.gl/RYAzoCFef6kyb2sb7,2024-08-07,agregados_adrian,100001,NaN,NaN,NaN,No se encontraron coordenadas


Los demas codigos(100001 a 100006) son referidos a ubicacion empresa y los tenemos en nuestro set **ubi_empresa.xlsx**

## BLOQUE 4: LIMPIEZA FINAL Y GUARDAD

In [23]:
# 4.1. Crear DataFrame final
print("\n4.1. Creando DataFrame final...")
ubi_cliente = df_ubicaciones[['CODIGO', 'NOMCLI', 'LATITUD', 'UBICACIÓN', 'LONGITUD', 'FECHA']].copy()

# 4.2. Conversiones finales
print("\n4.2. Realizando conversiones finales...")
ubi_cliente['LONGITUD'] = pd.to_numeric(ubi_cliente['LONGITUD'], errors='coerce')
ubi_cliente['LATITUD'] = pd.to_numeric(ubi_cliente['LATITUD'], errors='coerce')
ubi_cliente['NOMCLI'] = ubi_cliente['NOMCLI'].astype(str)
ubi_cliente['CODIGO'] = ubi_cliente['CODIGO'].astype('int64')

# 4.3. Limpiezas finales
print("\n4.3. Realizando limpiezas finales...")
filas_antes_limpieza = ubi_cliente.shape[0]
ubi_cliente = ubi_cliente.dropna(subset=['LATITUD', 'LONGITUD', 'UBICACIÓN', 'CODIGO', 'NOMCLI'])
ubi_cliente = ubi_cliente.drop_duplicates(subset=['NOMCLI'])
ubi_cliente.reset_index(drop=True, inplace=True)

# 4.4. Reportes finales
print("\n4.4. Generando reportes finales...")
zero_coords = ubi_cliente[(ubi_cliente['LATITUD'] == 0) & (ubi_cliente['LONGITUD'] == 0)]
print(f"Registros con coordenadas en 0: {len(zero_coords)}")

# 4.5. Guardar resultados
print("\n4.5. Guardando resultados...")
try:
    ruta_salida = "../Limpia/Ubicaciones_direcciones.xlsx"
    with pd.ExcelWriter(ruta_salida) as writer:
        ubi_cliente.to_excel(writer, sheet_name='Ubicaciones_Limpias', index=False)
        if not zero_coords.empty:
            zero_coords.to_excel(writer, sheet_name='Coordenadas_En_0', index=False)
    print(f"✓ Archivo guardado exitosamente en: {ruta_salida}")
except Exception as e:
    print(f"❌ Error al guardar: {e}")
    raise

print("\n=== PROCESO COMPLETADO ===")



4.1. Creando DataFrame final...

4.2. Realizando conversiones finales...

4.3. Realizando limpiezas finales...

4.4. Generando reportes finales...
Registros con coordenadas en 0: 0

4.5. Guardando resultados...
✓ Archivo guardado exitosamente en: ../Limpia/Ubicaciones_direcciones.xlsx

=== PROCESO COMPLETADO ===


In [24]:
df_ubicaciones.info()
df_ubicaciones.sample(5)

<class 'pandas.core.frame.DataFrame'>
Index: 745 entries, 0 to 745
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CODIGO             745 non-null    float64       
 1   NOMCLI             744 non-null    object        
 2   UBICACIÓN          745 non-null    object        
 3   FECHA              745 non-null    datetime64[ns]
 4   ORIGEN             745 non-null    object        
 5   CODCLI2            745 non-null    int64         
 6   TAREAS             497 non-null    float64       
 7   LATITUD            732 non-null    float64       
 8   LONGITUD           732 non-null    float64       
 9   RAZON_COORDENADAS  745 non-null    object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 64.0+ KB


,CODIGO,NOMCLI,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS,LATITUD,LONGITUD,RAZON_COORDENADAS
440,62906.0,DANIEL DOMINGUEZ,"http://maps.google.com/?q=-34.76591301171604,-...",2024-04-27,2,62906,62906.0,-34.765913,-55.755752,Coordenadas extraídas
654,41915.0,VERONICA 1 DE MAYO,"http://maps.google.com/?q=-34.75679234279331,-...",2024-04-27,2,41915,NaN,-34.756792,-55.775099,Coordenadas extraídas
44,80142.0,"80142,KIOSCO CRISTINA ( esta cerrado y creo q...",Marcador 76\n2024/10/30 @ 11:27:13\nhttp://map...,2024-11-04,agregados_adrian,80142,80142.0,-34.481296,-55.631477,Coordenadas extraídas
180,81232.0,"81232, Panadería el Reencuentro...Salinas Nort...","https://www.google.es/maps/place/Ruta+87,+1510...",2024-08-16,comentarios,81232,81232.0,-34.732611,-56.533716,Coordenadas extraídas
685,21393.0,CONDE,"http://maps.google.com/?q=-34.49006970915364,-...",2024-04-27,2,21393,21393.0,-34.490070,-55.566213,Coordenadas extraídas
